In [32]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gymnasium as gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Init the RSSM (including all networks)

In [33]:
rssm = RSSM()

Initializing encoder:
- adding ConvBlock((3, 64))                   ==> output shape: (64, 32, 32) ==> prod: 65536
- adding ConvBlock((64, 128))                   ==> output shape: (128, 16, 16) ==> prod: 32768
- adding ConvBlock((128, 256))                   ==> output shape: (256, 8, 8) ==> prod: 16384
- adding ConvBlock((256, 512))                   ==> output shape: (512, 4, 4) ==> prod: 8192
- adding ConvBlock((512, 256))                   ==> output shape: (256, 2, 2) ==> prod: 1024
- adding Flatten()
- adding Reshape: (*,1024) => (*,32,32)

Initializing decoder:
- adding Reshape: (*,1024) => (*,256,2,2)
- adding transpose ConvBlock(256, 256)                   ==> output shape: (256, 4, 4) ==> prod: 4096
- adding transpose ConvBlock(256, 512)                   ==> output shape: (512, 8, 8) ==> prod: 32768
- adding transpose ConvBlock(512, 256)                   ==> output shape: (256, 16, 16) ==> prod: 65536
- adding transpose ConvBlock(256, 128)                   ==> output shap

TODO:

required:
- use discrete actor critic agent [WIP ]
- distributional reward_predictor [ ]

optional:
(- output batches with rssm)
(- train on vector env with batches)

In [34]:
vae = CategoricalVAE()

Initializing encoder:
- adding ConvBlock((3, 64))                   ==> output shape: (64, 32, 32) ==> prod: 65536
- adding ConvBlock((64, 128))                   ==> output shape: (128, 16, 16) ==> prod: 32768
- adding ConvBlock((128, 256))                   ==> output shape: (256, 8, 8) ==> prod: 16384
- adding ConvBlock((256, 512))                   ==> output shape: (512, 4, 4) ==> prod: 8192
- adding ConvBlock((512, 256))                   ==> output shape: (256, 2, 2) ==> prod: 1024
- adding Flatten()
- adding Reshape: (*,1024) => (*,32,32)

Initializing decoder:
- adding Reshape: (*,1024) => (*,256,2,2)
- adding transpose ConvBlock(256, 256)                   ==> output shape: (256, 4, 4) ==> prod: 4096
- adding transpose ConvBlock(256, 512)                   ==> output shape: (512, 8, 8) ==> prod: 32768
- adding transpose ConvBlock(512, 256)                   ==> output shape: (256, 16, 16) ==> prod: 65536
- adding transpose ConvBlock(256, 128)                   ==> output shap

In [47]:
batch_size = 5
img_size = config["size"]
channels = 1 if config["grayscale"] else 3

input_tensor = torch.rand(batch_size, channels, *img_size)*255 # pixel values are in [0, 255]
input_tensor = input_tensor.to(config["device"]);

In [48]:
a, b = vae.encode(input_tensor)
a.shape, b.shape

(torch.Size([5, 32, 32]), torch.Size([5, 32, 32]))

In [41]:
input_tensor.device

device(type='cpu')

## Create the imagination environment for training the agent

In [3]:
replay_buffer = ReplayBuffer()

In [4]:
imagination_env = make_imagination_env(rssm, replay_buffer, render_mode=None)

Adding a TimeLimit wrapper with 16 max imagination episode steps.
Adding an AutoReset wrapper.
Adding a RescaleActionV0 wrapper. Low: [-1. -1. -1.], High: [1. 1. 1.]
Adding a Gymnasium RecordEpisodeStatistics wrapper.


## Init the agent

In [5]:
# agent = PPO(
#     policy="MlpPolicy",
#     env=imagination_env,
#     verbose=verbose,
#     tensorboard_log=log_dir,
#     gamma=gamma,
#     gae_lambda=lam,
#     ent_coef=ent_coef,
# )

agent = DiscreteActorCritic()

Initializing agent with 1536 features and 3 actions.
Initializing critic.
Adding zero weight init to the output layer.
Initializing actor.
Adding zero weight init to the output layer.


In [6]:
sample_instance = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])
sample_batch = torch.randn(32, config["H"] + config["Z"]).to(config["device"]) # torch.Size([32, 1536])

print(sample_instance.shape)
print(sample_batch.shape)

torch.Size([1536])
torch.Size([32, 1536])


In [7]:
batch_size = 5

In [8]:
action = torch.randn(batch_size, config["A"]).to(config["device"]) # torch.Size([3])
action.shape

torch.Size([5, 3])

In [9]:
h = torch.randn(batch_size, config["H"]).to(config["device"]) # torch.Size([512])
h.shape

torch.Size([5, 512])

In [10]:
z = torch.randn(batch_size, config["Z"]).to(config["device"]) # torch.Size([1024]) = 32*32
z.shape

torch.Size([5, 1024])

In [11]:
h, reward_pred, continue_prob, continue_pred, x_reconstruction = rssm.step(action, h, z)
print(h.shape)
print(reward_pred.shape)
print(continue_prob.shape)
print(continue_pred.shape)
print(x_reconstruction.shape)

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])


## Training loop


In [12]:
# agent_tracker = MetricsTracker(
#     training_metrics=["critic_loss", "actor_loss"],
#     episode_metrics=["rewards", "log_probs", "value_preds", "critic_dists", "entropies", "masks"],
# )
# 
# for imagination_phase in range(agent_update_phases_per_model_update):
#     
#     if imagination_phase == 0:
#         dream_obs, dream_info = imagination_env.reset(seed=42)
#         dream_obs = torch.tensor(dream_obs).to(device)
#     
#     for step in range(imagination_steps_per_agent_update):
#         
#         value_pred, critic_dist = agent.apply_critic(dream_obs)
# 
#         # Get an action and take an environment step
#         action, log_prob, actor_entropy = agent.get_action(dream_obs)
#         dream_obs, reward, terminated, truncated, info = imagination_env.step(to_np(action))
#         
#         # Transform the next obs
#         dream_obs = torch.Tensor(dream_obs).to(device)
#         
#         # every step:
#         agent_tracker.add(
#             episode_metrics={
#                 "rewards": reward,
#                 "log_probs": log_prob,
#                 "value_preds": value_pred,
#                 "critic_dists": critic_dist,
#                 "entropies": actor_entropy,
#                 "masks": torch.tensor(0.0 if terminated else 1.0),
#             }
#         )
#     
#     # every imagination phase:
#     episode_batches = agent_tracker.get_episode_batches() # episode_batches is a dict
#     last_value_pred, _ = agent.apply_critic(dream_obs) # last value prediction for GAE
# 
#     # Update the agent's parameters
#     critic_loss, actor_loss = agent.get_loss(episode_batches, last_value_pred)
#     agent.update_parameters(critic_loss, actor_loss)
#     
#     # Only for the last imagined episode: Log the episode metrics
#     if imagination_phase == agent_update_phases_per_model_update - 1:
# 
#         agent_tracker.add(
#             training_metrics={
#                 "critic_loss": critic_loss,
#                 "actor_loss": actor_loss,
#             }
#         )
#     
#         # Episode return
#         if len(imagination_env.return_queue):
#             agent_tracker.writer.add_scalar("dream_episode_return", np.array(imagination_env.return_queue)[-1], global_step=sample_phase)
#     
#         # Actor and critic losses
#         agent_tracker.log_to_tensorboard(step=sample_phase)
#     
#         

In [13]:
# New dreamer training loop with batches for the distributional critic
# (and later: add manual actor-critic training in imagination env. and delete sb3).
# add batches to rssm

### rssm.load_weights("weights/RSSM_1.70111713")
### rssm.train()

# Create the environment
env = make_env()
batch_size = config["n_envs"]

# World Model Logging
model_tracker = MetricsTracker(
    # Log the mean loss for the training metrics
    training_metrics=["loss", "image_loss", "reward_loss", "continue_loss", "dyn_loss", "rep_loss", "rewards"],
    
    # Loss per step
    episode_metrics=["loss", "image_loss", "reward_loss", "continue_loss", "dyn_loss", "rep_loss", "rewards"],
)

# Agent Logging
agent_tracker = MetricsTracker(
    training_metrics=["critic_loss", "actor_loss"],
    episode_metrics=["rewards", "log_probs", "value_preds", "critic_dists", "entropies", "masks"],
)


for sample_phase in tqdm(range(start_phase, n_seed_phases + n_model_updates)):
    
    # Reset the RNN's hidden state
    ### h = torch.zeros(rssm.num_rnn_layers, 1, H, device=device, dtype=torch.float32) # seq_len, B, H
    # NEW:
    # - init in rssm.step (if h is not given)
    
    if sample_phase == start_phase:
        
        # Get the first obs
        obs, info = env.reset(seed=42)
        x = transform(torch.tensor(obs)) # => (B,C,H,W)
        replay_buffer.push(x)
        
        # NEW:
        step_dict = rssm.pre_step(x=x)
    
    for step in range(n_steps_per_model_update):
    
        """ WORLD MODEL LEARNING """

        # Get real z and z prediction, also get state
        ###z_prior = rssm.dynamics_mlp(h).view(-1, num_categoricals, num_classes) # (1,32,32) for the softmax
        ###z_prior = F.softmax(z_prior, -1).flatten(start_dim=1, end_dim=2) # (1, 1024)
        ###z = rssm.vae.encode(x).flatten(start_dim=1, end_dim=2)
        ###state = to_np(torch.cat((h.flatten().detach(), z.flatten().detach()), dim=0))

        # Predict one step using the world model (for training it)
        ### h, reward_pred, continue_prob, continue_pred, x_pred = rssm.step(action, h, z)
        # NEW:
        h = step_dict["h"]
        step_dict = rssm.pre_step(h, x)
        assert h.shape == torch.Size([batch_size, H])
        assert x.shape == torch.Size([batch_size, 1 if grayscale else 3, *size])
        
        #for key, val in step_dict.items():
        #    print(f"{key} shape:", val.shape)
        
        # Get action (random in init phases, otherwise from agent)
        if sample_phase < n_seed_phases:
            action = env.action_space.sample()
        else:
            assert step_dict["state"].shape == torch.Size([batch_size, H+Z])
            action, _, _ = agent.get_action(step_dict["state"])
        
        # Update the world model state
        h = step_dict["h"]
        z = step_dict["z"]
        h = rssm.step(action, h, z)

        # Take an environment step with the action
        obs, reward, terminated, truncated, info = env.step(to_np(action))
        x = transform(torch.tensor(obs)) # => (B,C,H,W)
        replay_buffer.push(x)

        # Calculate the world model loss
        ### continue_target = torch.tensor(1 - (terminated | truncated), device=device, dtype=torch.float32)
        ### reward = torch.tensor(reward, device=device, dtype=torch.float32)
        ###losses = rssm.get_losses(x, x_pred, reward, reward_pred, 
        ###                         continue_target, continue_prob, z_prior, z)
        # NEW:
        step_dict["continue_target"] = continue_target = torch.tensor(1 - (terminated | truncated), device=device, dtype=torch.float32)
        step_dict["reward_target"] = reward = torch.tensor(reward, device=device, dtype=torch.float32)
        
        losses = rssm.get_losses(step_dict)

        # Track the losses
        model_tracker.add(
            episode_metrics=losses # losses is a dict with batches for all episode metrics
        )
        # Track the reward for the episode return
        model_tracker.add(
            episode_metrics={
                "rewards": reward,
            }
        )
    
    # Get mean loss and update world model
    episode_losses = model_tracker.get_episode_batches(reduction="mean") # episode_losses is a dict
    rssm.update_parameters(episode_losses["loss"])

    
    """ RL AGENT LEARNING (IN THE WORLD MODEL) """
    if verbose and sample_phase == n_seed_phases:
        print("The agent starts learning.")

        
    if sample_phase >= n_seed_phases:
        ### agent.learn(
        ###     total_timesteps=imagination_steps_per_model_update,
        ###     progress_bar=imagination_progress_bar,
        ###     reset_num_timesteps=False
        ### )
        
        # NEW: discrete actor critic
        for imagination_phase in range(agent_update_phases_per_model_update):
            print("IMAGINATION PHASE:", imagination_phase) # debug
            
            if imagination_phase == 0:
                dream_obs, dream_info = imagination_env.reset(seed=42)
                dream_obs = torch.tensor(dream_obs).to(device)

            for step in range(imagination_steps_per_agent_update):

                value_pred, critic_dist = agent.apply_critic(dream_obs)

                # Get an action and take an environment step
                action, log_prob, actor_entropy = agent.get_action(dream_obs)
                dream_obs, reward, terminated, truncated, info = imagination_env.step(to_np(action))

                # Transform the next obs
                dream_obs = torch.Tensor(dream_obs).to(device)

                # every step:
                agent_tracker.add(
                    episode_metrics={
                        "rewards": reward,
                        "log_probs": log_prob,
                        "value_preds": value_pred,
                        "critic_dists": critic_dist,
                        "entropies": actor_entropy,
                        "masks": torch.tensor(0.0 if terminated else 1.0),
                    }
                )

            # every imagination phase:
            episode_batches = agent_tracker.get_episode_batches() # episode_batches is a dict
            last_value_pred, _ = agent.apply_critic(dream_obs) # last value prediction for GAE

            # Update the agent's parameters
            critic_loss, actor_loss = agent.get_loss(episode_batches, last_value_pred)
            agent.update_parameters(critic_loss, actor_loss)

            # Only for the last imagined episode: Log the episode metrics
            print(imagination_phase, agent_update_phases_per_model_update-1)
            if imagination_phase == agent_update_phases_per_model_update - 1:
                print("HI")

                agent_tracker.add(
                    training_metrics={
                        "critic_loss": critic_loss,
                        "actor_loss": actor_loss,
                    }
                )

                # Episode return
                if len(imagination_env.return_queue):
                    agent_tracker.writer.add_scalar("dream_episode_return", np.array(imagination_env.return_queue)[-1], global_step=sample_phase)

                # Actor and critic losses
                agent_tracker.log_to_tensorboard(step=sample_phase)

    
    # Every couple episodes:
    if sample_phase % config["log_interval"] == 0:
        
        # Log: mean episode losses
        model_tracker.add(
            training_metrics=episode_losses
        )
        
        # Log: episode return
        if len(env.return_queue):
            model_tracker.writer.add_scalar("episode_return", np.array(env.return_queue)[-1], global_step=len(env.return_queue))
        
        # Tensorboard
        model_tracker.log_to_tensorboard(step=sample_phase)
        
        # TODO Later: actor and critic losses

        # Save original image and reconstruction
        save_image_and_reconstruction(step_dict["x"], step_dict["x_reconstruction"], sample_phase)

env.close()

    

Creating environment with properties:
- type: toy env
- adding a TimeLimit wrapper with 1000 max episode steps
- adding an AutoReset wrapper
- making a AsyncVectorEnv with 16 envs
- adding a RescaleActionV0 wrapper Low: -1, High: 1
- adding a Gymnasium RecordEpisodeStatistics wrapper


  0%|▍                                                                                      | 43/8812 [01:10<3:58:10,  1.63s/it]


KeyboardInterrupt: 

In [17]:
torch.rand(batch_size, 64,64,3).max()

tensor(1.0000)

In [ ]:
# 3 envs: 1.2/it
# 8 envs: 1.4/it
# 16 envs: 1.6/it

In [ ]:
plt.imshow(to_np(step_dict["x_reconstruction"][0].permute(1,2,0)))

In [ ]:
step_dict["x"].shape

In [ ]:
step_dict["reward_target"].shape

In [ ]:
step_dict["reward_target"]

In [ ]:
batch_size = 7
h = torch.randn(batch_size, config["H"]).to(config["device"])
x = torch.randn(batch_size, 3, 64, 64).to(config["device"]) #  (B, C, H, W)

In [ ]:
x.shape[2:4]

In [ ]:
torch.tensor(env.reset()[0][0]).shape
transform(torch.tensor(env.reset()[0][0])).shape

In [ ]:
step_dict = rssm.pre_step(x=x)
step_dict

In [ ]:
step_dict["z_pred"] == step_dict["z"]

In [ ]:
h = torch.zeros(batch_size, H)
h.shape

In [ ]:
PPO(
policy="MlpPolicy",
env=imagination_env,
verbose=verbose,
tensorboard_log=log_dir,
gamma=gamma,
gae_lambda=lam,
ent_coef=ent_coef).predict(state, deterministic=False)[0].shape

In [ ]:
action.shape

In [ ]:
action.squeeze().shape

In [ ]:
h.shape

In [ ]:
z.shape

In [ ]:
torch.cat((action.unsqueeze(0), h, z), 1).float()

In [ ]:
CategoricalStraightThrough.forward?

In [ ]:
# try:
# - training loop
# except KeyboardInterrupt:
#     """ Clean handling for interrupts to stop training early """
#     print("Stopping training.")
#     # Delete the last loss if the training was stopped early
#     # so that the list only consists of floats
#     for key in episode_losses:
#         if isinstance(episode_losses[key][-1], torch.Tensor):
#             episode_losses[key] = episode_losses[key][:-1]
# 
#     # Close the TensorBoard writer and the gym environment
#     writer.close()
#     env.close()

In [ ]:
#            # save the rssm and agent
#            rssm.save_weights(filename=f"RSSM_{best_running_loss:.8f}")
#            agent.save(f"weights/{agent.__class__.__name__}_agent")

## Plot the results

In [ ]:
plot_results = False

if plot_results:
    rolling_length = max(1, int(len(episode_losses["episode_loss"])/20))

    fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(3*5, 2*5))

    # Iterate over the keys and plot the losses
    for i, key in enumerate(episode_losses.keys()):
        row = i // 3
        col = i % 3

        axs[row, col].set_title(key)
        losses = episode_losses[key]
        losses_moving_average = (
            np.convolve(
                np.array(losses).flatten(), np.ones(rolling_length), mode="valid"
            )
            / rolling_length
        )
        axs[row, col].plot(range(len(losses)), losses, label=key)
        axs[row, col].plot(range(len(losses_moving_average)), losses_moving_average, label="moving average")
        axs[row, col].legend(loc="upper right")

    plt.tight_layout()
    plt.show()

## Showcase the trained agent playing in latent imagination

In [ ]:
showcase_agent = False

if showcase_agent:
    
    showcase_rewards = []
    imagination_env.render_mode = "gif"
    obs, info = imagination_env.reset()
    
    for i in range(500):
        
        # apply the RL agent in eval mode to get an action
        state = to_np(torch.cat((h.flatten().detach(), z.flatten().detach()), dim=0))
        action, _ = agent.predict(state, deterministic=False)
        # action = imagination_env.action_space.sample()
        
        obs, reward, terminated, truncated, info = imagination_env.step(action)
        showcase_rewards.append(reward)
        imagination_env.render()
        
        if terminated or truncated:
            break
        
    imagination_env.close()
    imagination_env.render_mode = None

    plt.plot(showcase_rewards)
    plt.show()

## Test area

In [ ]:
# !tensorboard 